## Setup project

In [1]:
!rm -rf Car-Parts-Segmentation/ project labels images dataset

In [2]:
!git clone https://github.com/sanayasfp/Car-Parts-Segmentation.git

Cloning into 'Car-Parts-Segmentation'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (564/564), done.
remote: Compressing objects: 100% (549/549), done.
remote: Total 564 (delta 27), reused 548 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (564/564), 24.04 MiB | 35.17 MiB/s, done.
Resolving deltas: 100% (27/27), done.


## Generate dataset

In [3]:
import os
import shutil
import json

In [4]:
def create_dataset_structure(path: str):
  dataset_path = os.path.join(path, "dataset")

  if not os.path.exists(dataset_path):
    os.makedirs(dataset_path, True)

  images_path = os.path.join(dataset_path, "images")
  labels_path = os.path.join(dataset_path, "labels")

  if not os.path.exists(images_path):
    os.makedirs(images_path, True)

  if not os.path.exists(labels_path):
    os.makedirs(labels_path, True)

  for path in [images_path, labels_path]:
    for d in ["train", "val", "test"]:
      if not os.path.exists(os.path.join(path, d)):
        os.makedirs(os.path.join(path, d), True)

  return dataset_path, images_path, labels_path

In [5]:
def write_yolo_labels(anotations_for_image, categories, image_name, image_width, image_height, labels_path):
  file_name = image_name.split(".")[0]

  with open(os.path.join(labels_path, "{}.txt".format(file_name)), "w") as f:
    for index, annotation in enumerate(anotations_for_image):
      category_id = annotation["category_id"]
      category = next(filter(lambda x: x["id"] == category_id, categories))
      class_name = category["name"]
      class_id = category["id"] -2

      if class_id < 0:
        continue

      bbox = annotation["bbox"]

      x_center = (bbox[0] + bbox[2] / 2) / image_width
      y_center = (bbox[1] + bbox[3] / 2) / image_height
      width = bbox[2] / image_width
      height = bbox[3] / image_height

      newline = "\n" if index != len(anotations_for_image) - 1 else ""

      f.write("{} {} {} {} {}{}".format(class_id, x_center, y_center, width, height, newline))

    f.close()
    return class_id, class_name

In [6]:
def anotate_image(dataset_path, image_dest_path: str, labels_path: str, part: str = "train"):
  anotation_file_path = os.path.join(dataset_path, "annotations.json")

  if not os.path.exists(anotation_file_path):
    raise ValueError(f'File {anotation_file_path} does not exist')

  model_class: set(tuple) = set()

  with open(anotation_file_path, "r") as f:
    anotation_data = json.load(f)

    for image in anotation_data["images"]:
      image_name = image["file_name"]
      image_id = image["id"]
      image_width = image["width"]
      image_height = image["height"]
      image_path = os.path.join(dataset_path, image["path"])
      anotations_for_image = [a for a in anotation_data["annotations"] if a["image_id"] == image_id]

      if not os.path.exists(image_path):
        continue

      shutil.copy2(image_path, os.path.join(image_dest_path, part, image_name))
      class_id, class_name = write_yolo_labels(
          anotations_for_image,
          anotation_data["categories"],
          image_name,
          image_width,
          image_height,
          os.path.join(labels_path, part)
      )
      model_class.add((class_id, class_name))

    f.close()
    return model_class

In [7]:
def sort_model_class(model_class):
  return sorted(model_class, key=lambda x: x[0])

In [8]:
def generate_dataset(path: str = "."):
  _, images_path, labels_path = create_dataset_structure(path)
  trainset_path = "Car-Parts-Segmentation/trainingset"
  testset_path = "Car-Parts-Segmentation/testset"

  dataset_part = ["train", "test"]
  dataset = {}

  for index, path in enumerate([trainset_path, testset_path]):
    model_class = anotate_image(path, images_path, labels_path, dataset_part[index])
    dataset[dataset_part[index]] = sort_model_class(model_class)

  return dataset

In [9]:
dataset_class = generate_dataset("project")

## Generate Config file (config.yml)

In [10]:
def generate_dataset_config(dataset_class: dict, path: str = "."):
  dataset_path = os.path.join(os.path.abspath(path), "dataset")
  config_path = os.path.join(dataset_path, "config.yaml")
  part = dataset_class.keys()

  if not "train" in part:
    raise ValueError("Train set not found")

  model_class = dataset_class['train']
  train = "images/train"
  val = "images/val" if "val" in part else train
  test = "images/test" if "test" in part else val

  with open(config_path, "w") as f:
    f.write(f"path: {dataset_path}\n")
    f.write(f"train: {train}\n")
    f.write(f"val: {val}\n")
    f.write(f"test: {test}\n")
    f.write("nc: {}\n".format(len(model_class)))
    f.write("names: \n")

    for class_id, class_name in model_class:
      f.write(f"  {class_id}: {class_name}\n")

    f.close()

In [11]:
generate_dataset_config(dataset_class, "./project")

## Train model

In [12]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 21.6 MB/s eta 0:00:00


In [13]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(data="/content/project/dataset/config.yaml", epochs=1)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 97.6MB/s]


Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/project/dataset/config.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

100%|██████████| 755k/755k [00:00<00:00, 24.8MB/s]


Overriding model.yaml nc=80 with nc=18

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.32G      1.434      4.355      1.545        202        640: 100%|██████████| 25/25 [00:09<00:00,  2.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08<00:00,  1.61it/s]


                   all        400       3073     0.0624       0.04     0.0358     0.0281

1 epochs completed in 0.007 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,009,158 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.99it/s]


                   all        400       3073     0.0623       0.04     0.0358      0.028
           back_bumper        101        101      0.143     0.0099     0.0724     0.0724
            back_glass        112        112          0          0          0          0
        back_left_door        122        122          0          0          0          0
       back_left_light        149        149          0          0          0          0
       back_right_door        103        103          0          0          0          0
      back_right_light        119        119          0          0          0          0
          front_bumper        239        239      0.129      0.661      0.123     0.0664
           front_glass        218        218        0.4    0.00917      0.202      0.162
       front_left_door        132        132          0          0          0          0
      front_left_light        236        236          0          0          0          0
      front_right_doo

In [15]:
results.

AttributeError: 'DetMetrics' object has no attribute 'save'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (dict of str): A dict of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots path and data when th

## Validate model

In [14]:
!yolo task=detect \
mode=val \
model=./runs/detect/train/weights/best.pt \
data=./project/dataset/config.yaml

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

Predict with model

In [ ]:
!yolo task=detect \
mode=predict \
model=./runs/detect/train/weights/best.pt \
conf=0.25 \
source=./project/dataset/images/test/

In [ ]:
!yolo task=detect \
mode=predict \
model=./runs/detect/train/weights/best.pt \
conf=0.25 \
source=./project/ressources/front.jpeg